# Notebook 1: Carga de Datos

## Objetivo
Este notebook se encarga de la ingesta inicial de datos históricos de precios, validación básica de calidad y definición del universo de inversión según las especificaciones del proyecto.

## Índice
1. [Configuración del Entorno](#configuracion)
2. [Carga de Datos Históricos](#carga-datos)
3. [Validación y Limpieza Inicial](#validacion)
4. [Definición del Universo de Inversión](#universo)
5. [Guardado de Datos Preparados](#guardado)

---

## 1. Configuración del Entorno {#configuracion}

Importación de librerías permitidas y configuración inicial del entorno de trabajo.

In [ ]:
# Librerías permitidas según especificaciones
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime, timedelta
import warnings
import os

# Configuración
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("Librerías importadas correctamente")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Carga de Datos Históricos {#carga-datos}

Carga del archivo histórico de precios desde Google Drive o fuente local.

**Nota:** Según el PDF, el histórico está disponible en:
https://drive.google.com/file/d/1nvubXdAu0EONlrP_yrURZbnPhBQ-uDaB/view?usp=sharing

In [ ]:
def load_price_data(file_path=None, use_yfinance=False, symbols=None, start_date='2014-01-01'):
    """
    Carga datos históricos de precios.
    
    Parámetros:
    -----------
    file_path : str, optional
        Ruta al archivo local (CSV, Parquet, etc.)
    use_yfinance : bool
        Si True, descarga datos usando yfinance
    symbols : list, optional
        Lista de símbolos a descargar (si use_yfinance=True)
    start_date : str
        Fecha de inicio para descarga (formato 'YYYY-MM-DD')
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con precios históricos (índice: fecha, columnas: símbolos)
    """
    if file_path and os.path.exists(file_path):
        # Carga desde archivo local
        if file_path.endswith('.parquet'):
            df = pd.read_parquet(file_path)
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path, index_col=0, parse_dates=True)
        else:
            raise ValueError(f"Formato no soportado: {file_path}")
        print(f"Datos cargados desde: {file_path}")
        
    elif use_yfinance and symbols:
        # Descarga usando yfinance
        print(f"Descargando datos para {len(symbols)} símbolos desde {start_date}...")
        data_dict = {}
        for symbol in symbols:
            try:
                ticker = yf.Ticker(symbol)
                hist = ticker.history(start=start_date)
                if not hist.empty:
                    data_dict[symbol] = hist['Close']
                    print(f"  ✓ {symbol}: {len(hist)} días")
                else:
                    print(f"  ✗ {symbol}: Sin datos")
            except Exception as e:
                print(f"  ✗ {symbol}: Error - {str(e)}")
        
        df = pd.DataFrame(data_dict)
        print(f"\nTotal de símbolos cargados: {len(df.columns)}")
    else:
        raise ValueError("Debe proporcionar file_path o usar use_yfinance=True con symbols")
    
    return df


# CARGAR DATOS AQUÍ
# Opción 1: Desde archivo local (ajustar ruta según tu caso)
# price_data = load_price_data(file_path='../data/historical_prices.parquet')

# Opción 2: Desde Google Drive (descargar manualmente primero)
# price_data = load_price_data(file_path='../data/historical_prices.csv')

# Opción 3: Usando yfinance (solo para pruebas, no recomendado para todo el universo)
# symbols_sp500 = ['SPY', 'AAPL', 'MSFT', ...]  # Lista completa del S&P 500
# price_data = load_price_data(use_yfinance=True, symbols=symbols_sp500, start_date='2014-01-01')

# EJEMPLO: Cargar datos (REEMPLAZAR CON TU MÉTODO)
print("\n⚠️  IMPORTANTE: Reemplazar esta celda con la carga real de datos")
print("El archivo debe contener precios de cierre diarios del S&P 500 desde 2014")

# Estructura esperada del DataFrame:
# - Índice: fechas (datetime)
# - Columnas: símbolos de activos (ej: 'AAPL', 'MSFT', 'SPY', etc.)
# - Valores: precios de cierre (float)

## 3. Validación y Limpieza Inicial {#validacion}

Validación de calidad de datos: fechas, valores faltantes, duplicados y coherencia temporal.

In [ ]:
def validate_price_data(df, min_date='2014-01-01'):
    """
    Valida y limpia datos de precios.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con precios históricos
    min_date : str
        Fecha mínima requerida
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame validado y limpio
    dict
        Diccionario con estadísticas de validación
    """
    validation_stats = {
        'fecha_inicio': None,
        'fecha_fin': None,
        'total_dias': 0,
        'total_activos': 0,
        'activos_con_datos': 0,
        'valores_faltantes_pct': 0.0,
        'duplicados': 0,
        'valores_negativos': 0
    }
    
    # Validar índice de fechas
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    
    df = df.sort_index()
    
    # Estadísticas básicas
    validation_stats['fecha_inicio'] = df.index.min()
    validation_stats['fecha_fin'] = df.index.max()
    validation_stats['total_dias'] = len(df)
    validation_stats['total_activos'] = len(df.columns)
    
    # Validar fechas mínimas
    if validation_stats['fecha_inicio'] > pd.to_datetime(min_date):
        warnings.warn(f"Los datos comienzan en {validation_stats['fecha_inicio']}, se requiere desde {min_date}")
    
    # Detectar duplicados
    validation_stats['duplicados'] = df.index.duplicated().sum()
    if validation_stats['duplicados'] > 0:
        df = df[~df.index.duplicated(keep='first')]
        print(f"⚠️  Eliminados {validation_stats['duplicados']} días duplicados")
    
    # Validar valores
    validation_stats['valores_negativos'] = (df < 0).sum().sum()
    if validation_stats['valores_negativos'] > 0:
        print(f"⚠️  Detectados {validation_stats['valores_negativos']} valores negativos")
    
    # Estadísticas de valores faltantes
    missing_pct = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
    validation_stats['valores_faltantes_pct'] = df.isna().sum().sum() / (len(df) * len(df.columns)) * 100
    
    # Activos con datos suficientes
    validation_stats['activos_con_datos'] = (missing_pct < 50).sum()
    
    return df, validation_stats


# Ejecutar validación (descomentar cuando price_data esté cargado)
# price_data_clean, validation_stats = validate_price_data(price_data, min_date='2014-01-01')

# Mostrar estadísticas
# print("\n=== ESTADÍSTICAS DE VALIDACIÓN ===")
# for key, value in validation_stats.items():
#     print(f"{key}: {value}")

# print("\n=== PRIMERAS FILAS ===")
# print(price_data_clean.head())

# print("\n=== ÚLTIMAS FILAS ===")
# print(price_data_clean.tail())

print("⚠️  Ejecutar validación después de cargar price_data")

In [ ]:
def define_investment_universe(df, lookback_months=13, min_trading_days=0.8):
    """
    Define el universo de inversión para cada mes.
    
    Criterios:
    - Activos que forman parte del S&P 500 durante últimos 13 meses
    - Mínimo de días de cotización (para evitar activos con muchos faltantes)
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con precios históricos
    lookback_months : int
        Meses hacia atrás para considerar activo elegible
    min_trading_days : float
        Proporción mínima de días con datos (0.8 = 80%)
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con universo mensual (índice: fecha, columnas: activos elegibles)
    """
    # Resample a mensual (último día hábil del mes)
    monthly_prices = df.resample('M').last()
    
    # Para cada mes, determinar activos elegibles
    universe_dict = {}
    
    for date in monthly_prices.index:
        # Ventana de lookback
        start_date = date - pd.DateOffset(months=lookback_months)
        window_data = df.loc[start_date:date]
        
        # Calcular proporción de datos disponibles por activo
        data_availability = (window_data.notna().sum() / len(window_data))
        
        # Filtrar activos con suficiente disponibilidad
        eligible_assets = data_availability[data_availability >= min_trading_days].index.tolist()
        
        universe_dict[date] = eligible_assets
    
    # Crear DataFrame con universo mensual
    universe_df = pd.DataFrame.from_dict(universe_dict, orient='index')
    universe_df.index.name = 'fecha'
    
    return universe_df, monthly_prices


# Ejecutar definición de universo (descomentar cuando price_data_clean esté disponible)
# universe_monthly, prices_monthly = define_investment_universe(
#     price_data_clean, 
#     lookback_months=13, 
#     min_trading_days=0.8
# )

# print("\n=== UNIVERSO MENSUAL ===")
# print(f"Total de meses: {len(universe_monthly)}")
# print(f"Promedio de activos elegibles por mes: {universe_monthly.apply(lambda x: len([a for a in x if pd.notna(a)]), axis=1).mean():.0f}")

# print("\n=== EJEMPLO: Primer mes ===")
# first_month = universe_monthly.index[0]
# print(f"Fecha: {first_month}")
# print(f"Activos elegibles: {len([a for a in universe_monthly.loc[first_month] if pd.notna(a)])}")

print("⚠️  Ejecutar definición de universo después de validar datos")

## 5. Guardado de Datos Preparados {#guardado}

Guardado de datos procesados para uso en notebooks posteriores.

In [ ]:
# Crear directorio de datos si no existe
data_dir = '../data'
os.makedirs(data_dir, exist_ok=True)

# Guardar datos procesados (descomentar cuando estén disponibles)
# price_data_clean.to_parquet(f'{data_dir}/price_data_clean.parquet')
# prices_monthly.to_parquet(f'{data_dir}/prices_monthly.parquet')

# Guardar universo mensual como CSV para fácil lectura
# universe_monthly.to_csv(f'{data_dir}/universe_monthly.csv')

print("\n=== RESUMEN DEL NOTEBOOK 1 ===")
print("✓ Configuración del entorno")
print("✓ Funciones de carga y validación creadas")
print("✓ Estructura lista para procesar datos")
print("\n⚠️  IMPORTANTE: Ejecutar todas las celdas con datos reales")
print("⚠️  Los datos deben guardarse en formato Parquet para eficiencia")